# Interacting with the FRBViewer widget

This widget ports the functionality of the yt pixelization function to Rust, and then to WebAssembly, so that the actual change in the image is conducted in the browser. Once the display has been created, no further contact with the server is necessary to get new data! It will pass some info back and forth (such as bounds and the like) but all of the actual visualization is occuring locally, in the browser, and is thus much faster with lower latency.

It may appear blank at first, in which case you will need to toggle one of the viewable parameters, such as the colormap log scaling, to ensure an image appears.

First, let's import our necessary modules

In [ ]:
import ipywidgets

import widgyts
import yt

Now let's use yt to load in some data. 

In [ ]:
ds = yt.load("enzo_tiny_cosmology/DD0046/DD0046")

## Slices and Projections

Now we'll take a slice of our dataset and pass that slice into the FRBViewer widget from widgyts

In [ ]:
# making the slice
s = ds.r[:, :, 0.5]

# putting data into a FRBViewer (loading the data into wasm)
frb_dens = widgyts.FRBViewer(
    height=512,
    width=512,
    px=s["px"],
    py=s["py"],
    pdx=s["pdx"],
    pdy=s["pdy"],
    val=s["density"],
    layout=ipywidgets.Layout(width="500px", height="500px"),
)

# using the default controls menu
controls = frb_dens.setup_controls()
display(ipywidgets.HBox([controls, frb_dens]))

Similarly, we can use the `proj(field, axis)` operation on the dataset to get a projection object of the density projected on the z axis and pass that to the FRB. 

**note**: to see this image, you'll want to go into the colormap dropdown and set the log scale of the colorbar. You can zoom in or out on the image as well. 

In [ ]:
m = ds.proj("density", "z")

frb_proj_dens = widgyts.FRBViewer(
    height=512,
    width=512,
    px=m["px"],
    py=m["py"],
    pdx=m["pdx"],
    pdy=m["pdy"],
    val=m["density"],
    layout=ipywidgets.Layout(width="500px", height="500px"),
)

# using the default controls menu
controls = frb_proj_dens.setup_controls()
display(ipywidgets.HBox([controls, frb_proj_dens]))

## Updating the Traitlets

### Traitlets in the colormap widget

The API documentation for widgyts details a bit about the traitlets that each widget has. The control panel on the left side of these earlier visualizations directly interact with some of those traitlets. You can use the controls to update these values, but you can also modify them directly. 

This next cell will update the density projection image's colormap to inferno, change the scale to log if it isn't already, and also update teh colormap bounds (the min and max value defaults for this dataset are ~1.3e-05 and 5.6e-02, respectively).

Note that the traitlets in this cell belong to the colormaps class. All colormap-related traitlets are handled by the Colormap widget. A few cells down we can also see some examples of updating traitlets that belong to the FRBViewer widget. 

In [ ]:
frb_proj_dens.colormaps.map_name = "inferno"
frb_proj_dens.colormaps.is_log = True
frb_proj_dens.colormaps.min_val = 1.2e-05
frb_proj_dens.colormaps.max_val = 5.0e-02


display(frb_proj_dens)

Note here that the display of the frb_proj_dens updated in this cell and the previous cell with the navigation panel. 

### Linking views with traitlets 

Updating traitlets directly is helpful in many ways. We can also use ipywidgets traitlet linking to make two views sync together. First, let's make another projection, but this time with temperature. 

In [ ]:
n = ds.proj("temperature", "z")

frb_proj_temp = widgyts.FRBViewer(
    height=512,
    width=512,
    px=n["px"],
    py=n["py"],
    pdx=n["pdx"],
    pdy=n["pdy"],
    val=n["temperature"],
    layout=ipywidgets.Layout(width="500px", height="500px"),
)

Now let's link the traitlets between the density and temperature projections and set the temperature colormap to be something different from density 

In [ ]:
ipywidgets.link((frb_proj_dens, "view_center"), (frb_proj_temp, "view_center"))
ipywidgets.link((frb_proj_dens, "view_width"), (frb_proj_temp, "view_width"))
ipywidgets.link(
    (frb_proj_dens.colormaps, "is_log"), (frb_proj_temp.colormaps, "is_log")
)

frb_proj_temp.colormaps.map_name = "magma"
frb_proj_temp.colormaps.min_val = 1.59e29
frb_proj_temp.colormaps.max_val = 1.125e32

Let's set up a view with a control panel and both the temperature and projection views.

If you update aspects of the view parameters of the density plot, the temperature plot will update in the same manner. Try and click within either of the images to re-center it, or use the zoom button to see interesting features of either map. 

In [ ]:
controls = frb_proj_dens.setup_controls()
display(ipywidgets.HBox([controls, frb_proj_dens, frb_proj_temp]))

Note here that the control panel is linked to the density projection plot, but you can still update the temperature plot using the traitlets. 

In [ ]:
frb_proj_temp.colormaps.map_name = "inferno"